# Stock Portfolio Optimization: Modern Portfolio Theory
**Description**: Optimizing an investment portfolio based on a user's risk tolerance: portfolio weighted average volatility, portfolio weighted average beta, sector diversification constraints. 

**User base**: Any retail investor 

**Purposes**: Can be used for Roth IRA investment strategy, structuring custom portfolios, a learning about risk's effect on returns

**Limitations**: 
- Historical performance is NOT an indicator of future performance. 
- While each asset's beta and volatility are based on historical data (valid assumptions), expected returns are based on random variables associated with the asset industry's past performance and volatility.
- Portfolio volatility cannot be used in the analysis due to computational complexity associated with covariances of all feasible asset pairs

## Data Collection and Cleaning

#### Part 1: Data Collection

**Data Sources**

- *"stock_data_SPY.csv"*: Google Finance was used (through Google sheets) to collect weekly closing prices for stocks in the S&P 500. Only populated 286 stocks with full weekly closing price information. 

- *"stock_sectors.csv"*: Sector source url: "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies". 

In [340]:
import gams
import numpy as np
import pandas as pd

# Reading in file data
stock_data_spy = pd.read_csv("complete_stock_data_SPY.csv")
stock_sectors = pd.read_csv("stock_sectors.csv")

# transform date
stock_data_spy['Date'] = pd.to_datetime(stock_data_spy['Date'])
stock_data_spy.set_index('Date', inplace=True)

# Calculating the necessary metrics
# Weekly returns
weekly_returns = stock_data_spy.pct_change().dropna()

# Annualized Average Return
years = (stock_data_spy.index[-1] - stock_data_spy.index[0]).days / 365.25
annualized_avg_return = ((stock_data_spy.iloc[-1] / stock_data_spy.iloc[0]) ** (1/years)) - 1

# Annualized Volatility
annualized_volatility = weekly_returns.std() * (52 ** 0.5)

# Beta Values
cov_matrix = weekly_returns.cov()
market_variance = weekly_returns['SPY'].var()
beta_values = cov_matrix.loc[:, 'SPY'] / market_variance

# Compiling the data
data_metrics = pd.DataFrame({
    'Annualized Average Return': annualized_avg_return,
    'Annualized Volatility': annualized_volatility,
    'Beta Value': beta_values
})

# Joining with the sectors data
stock_sectors.set_index(stock_sectors.columns[0], inplace=True)
complete_data = data_metrics.join(stock_sectors, how='inner')
complete_data.head()


,Annualized Average Return,Annualized Volatility,Beta Value,Sector
MMM,-0.136077,0.259216,0.865184,Industrials
ABT,0.080846,0.260149,0.755388,Health Care
ACN,0.163710,0.275473,1.085449,Information Technology
ADBE,0.212653,0.349557,1.164405,Information Technology
AMD,0.448503,0.504404,1.498770,Information Technology


### Sector Summaries

In [341]:
sector_stats = complete_data.groupby(stock_sectors.columns[0]).agg({
    'Annualized Average Return': 'mean',
    'Annualized Volatility': 'mean',
    'Beta Value': 'mean'
}).rename(columns={
    'Annualized Average Return': 'Average Return',
    'Annualized Volatility': 'Average Volatility',
    'Beta Value': 'Average Beta'
})

# Counting the total stocks in each sector
sector_stats['Total Stocks'] = complete_data.groupby(stock_sectors.columns[0]).size()

sector_stats

,Average Return,Average Volatility,Average Beta,Total Stocks
Sector,,,,
Communication Services,0.062548,0.307386,0.908172,9
Consumer Discretionary,0.079018,0.393833,1.325326,29
Consumer Staples,0.043583,0.263286,0.626993,28
Energy,0.084867,0.471914,1.156946,18
Financials,0.056629,0.352514,1.202321,47
Health Care,0.080841,0.296566,0.817876,29
Industrials,0.117397,0.313363,1.065299,41
Information Technology,0.165214,0.349312,1.140195,33
Materials,0.063187,0.364603,1.093505,16


### Data Preparation

**GAMS API Preparation**

- Expected returns calculated using historical sector average annual return * normal distribution RV * diminished asset volatility (normalize and standardize returns)

- Factorized the Sectors in preparation for the GAMS API

In [342]:
returns = sector_stats["Average Return"].to_dict()
volatilities = sector_stats["Average Volatility"].to_dict()
complete_data['Sector Average Return'] = complete_data[' Sector'].map(returns)
complete_data['Sector Average Volatility'] = complete_data[' Sector'].map(volatilities)
np.random.seed(42)

# Calculating new expected return for each asset
final_stocks = complete_data.copy()
final_stocks['expected_return'] = final_stocks['Sector Average Return'] + \
                                  np.random.normal(size=len(final_stocks)) * final_stocks['Sector Average Volatility'] / 10

# Selecting and renaming the required columns
final_stocks = final_stocks[['expected_return', 'Annualized Volatility', 'Beta Value', ' Sector']]
final_stocks.rename(columns={
    'Annualized Volatility': 'volatility',
    'Beta Value': 'beta',
    ' Sector': ' Sector'
}, inplace=True)

# Factorizing the Sector column for GAMS API preparation
final_stocks['Sector_factor'] = pd.factorize(final_stocks[' Sector'])[0]

final_stocks.index.name = 'Stock'
final_stocks.index

Index(['MMM', 'ABT', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'AKAM',
       ...
       'WDC', 'WY', 'WHR', 'WMB', 'WEC', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZION'],
      dtype='object', name='Stock', length=286)

**Correlation Matrix Preparation** 

- Some stocks were not able to be matched up with sector, so some data lost
- Calculate the correlation matrix for between each asset (used in the portfolio variance calculations)

In [343]:
weekly_returns_reset = weekly_returns.reset_index()
final_weekly_returns = weekly_returns_reset[final_stocks.index]
corr_matrix = final_weekly_returns.corr()
corr_matrix

,MMM,ABT,ACN,ADBE,AMD,AES,AFL,A,APD,AKAM,...,WDC,WY,WHR,WMB,WEC,WYNN,XEL,XYL,YUM,ZION
MMM,1.000000,0.430483,0.550162,0.412350,0.341070,0.514180,0.566818,0.505864,0.482324,0.381664,...,0.536511,0.550847,0.568439,0.506430,0.389397,0.465813,0.409447,0.660621,0.485623,0.488627
ABT,0.430483,1.000000,0.492621,0.461117,0.323065,0.349297,0.384281,0.589219,0.406689,0.481428,...,0.286549,0.457994,0.471614,0.307854,0.495739,0.289177,0.499166,0.464288,0.520469,0.192200
ACN,0.550162,0.492621,1.000000,0.639912,0.501774,0.571887,0.544641,0.620607,0.505188,0.538812,...,0.458394,0.618165,0.553472,0.412339,0.413559,0.435405,0.440126,0.591189,0.579414,0.456854
ADBE,0.412350,0.461117,0.639912,1.000000,0.542684,0.327634,0.304430,0.559861,0.377009,0.522133,...,0.387812,0.447822,0.473202,0.233099,0.259093,0.319229,0.289783,0.401408,0.362863,0.263275
AMD,0.341070,0.323065,0.501774,0.542684,1.000000,0.274349,0.334777,0.451998,0.367287,0.366011,...,0.446418,0.410199,0.425945,0.283875,0.270680,0.339220,0.273834,0.366960,0.365241,0.263718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WYNN,0.465813,0.289177,0.435405,0.319229,0.339220,0.482326,0.585693,0.429312,0.450587,0.211416,...,0.558846,0.626613,0.498505,0.549681,0.294752,1.000000,0.303091,0.558087,0.616037,0.538590
XEL,0.409447,0.499166,0.440126,0.289783,0.273834,0.470881,0.553465,0.370254,0.427050,0.371013,...,0.311516,0.570943,0.478677,0.496891,0.928252,0.303091,1.000000,0.507561,0.616911,0.227572
XYL,0.660621,0.464288,0.591189,0.401408,0.366960,0.610608,0.628321,0.610214,0.572488,0.289178,...,0.466812,0.608315,0.614115,0.516222,0.472682,0.558087,0.507561,1.000000,0.647973,0.481665
YUM,0.485623,0.520469,0.579414,0.362863,0.365241,0.554196,0.714033,0.486857,0.524877,0.343641,...,0.504280,0.707816,0.622193,0.554189,0.623487,0.616037,0.616911,0.647973,1.000000,0.407209


In [197]:
def get_portfolio_insights(portfolio_records, data):
    selected_stocks = portfolio_records[portfolio_records['level'] > 0]

    final_portfolio_df = pd.merge(data, selected_stocks, left_on='Stock', right_on='Stock', how='inner')
    final_portfolio_df = final_portfolio_df[['Stock', 'level', ' Sector', 'expected_return', 'volatility', 'beta']]

    final_portfolio_df.rename(columns={'level': '% Portfolio'}, inplace=True)
    final_portfolio_df['% Portfolio'] *= 100

    # Calculate weighted averages
    total_percentage = final_portfolio_df['% Portfolio'].sum()
    weighted_avg_return = (final_portfolio_df['expected_return'] * final_portfolio_df['% Portfolio']).sum() / total_percentage
    weighted_avg_volatility = (final_portfolio_df['volatility'] * final_portfolio_df['% Portfolio']).sum() / total_percentage
    weighted_avg_beta = (final_portfolio_df['beta'] * final_portfolio_df['% Portfolio']).sum() / total_percentage

    # Create summary row
    summary_row = pd.DataFrame([['PORTFOLIO', total_percentage, 'NA', weighted_avg_return, weighted_avg_volatility, weighted_avg_beta]], 
                               columns=['Stock', '% Portfolio', ' Sector', 'expected_return', 'volatility', 'beta'])
    
    # Append summary row
    final_portfolio_df = pd.concat([final_portfolio_df, summary_row], ignore_index=True)

    return final_portfolio_df

## Optimization Using GAMS

Open the GAMS container

In [344]:
%reload_ext gams.magic
m = gams.exchange_container

--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).


Load in scalar constraints

In [345]:
%%gams
scalars 
    beta_max /.9/,
    vol_max /.4/,
    max_sector /.20/,
    min_sector /.05/;

--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).


In [359]:
flattened_corr_matrix = [(row_index, col_index, corr_matrix.iloc[row_index, col_index])
                         for row_index in range(len(corr_matrix))
                         for col_index in range(len(corr_matrix.columns))]
# final_stocks.index
flattened_corr_matrix[0:5]

[(0, 0, 1.0),
 (0, 1, 0.43048286280481896),
 (0, 2, 0.5501622209338572),
 (0, 3, 0.4123498823912243),
 (0, 4, 0.3410700917237069)]

Add the parameters from the pandas data frame

In [356]:
ticker = m.addSet('Stock', records=final_stocks.index)
Sector_set = m.addSet('Sector', records=sector_stats.index)

expected_return = m.addParameter('Expected_return', [ticker], records=final_stocks['expected_return'])
volatility = m.addParameter('Volatility', [ticker], records=final_stocks['volatility'])
beta_v = m.addParameter('beta_v', [ticker], records=final_stocks['beta'])
sector = m.addParameter('Sector_v', [ticker], records=final_stocks['Sector_factor'])

portfolio = m.addVariable('Portfolio', 'positive', [ticker])
portfolio_return = m.addVariable('Portfolio_return')

CorrelationMatrix = m.addParameter('CorrelationMatrix', [ticker, ticker], records=flattened_corr_matrix)

display(CorrelationMatrix.records)

,Stock_0,Stock_1,value
0,0,0,1.000000
1,0,1,0.430483
2,0,2,0.550162
3,0,3,0.412350
4,0,4,0.341070
...,...,...,...
81791,285,281,0.538590
81792,285,282,0.227572
81793,285,283,0.481665
81794,285,284,0.407209


Add variables and equations

In [309]:
%%gams
Equations objective, total_amt, vol_cons, beta_cons, sector_cons_max(Sector), sector_cons_min(Sector);

objective..
    portfolio_return =e= sum(Stock, expected_return(Stock) * portfolio(Stock));

total_amt..
    1 =g= sum(Stock, portfolio(Stock));

vol_cons..
    vol_max =g= sum(Stock, volatility(Stock) * portfolio(Stock));

beta_cons..
    beta_max =g= sum(Stock, beta_v(Stock) * portfolio(Stock));

sector_cons_max(Sector)..
    max_sector =g= sum((Stock)$(Sector_v(Stock) = ord(Sector)-1), portfolio(Stock));

sector_cons_min(Sector)..
    min_sector =l= sum((Stock)$(Sector_v(Stock) = ord(Sector)-1), portfolio(Stock));

model portfolio_opt /all/;
solve portfolio_opt using lp maximizing portfolio_return;

--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.1766,26,287,LP,CPLEX,0


In [310]:
get_portfolio_insights(portfolio.records, final_stocks)

,Stock,% Portfolio,Sector,expected_return,volatility,beta
0,BMY,5.000000,Health Care,0.135773,0.231939,0.547158
1,CPB,5.000000,Consumer Staples,0.065239,0.247945,0.241151
2,CCI,5.000000,Real Estate,0.050696,0.288169,0.835119
3,EA,5.000000,Communication Services,0.078325,0.270202,0.567523
4,GE,20.000000,Industrials,0.194586,0.401787,1.153059
5,IBM,20.000000,Information Technology,0.219356,0.262016,0.839218
6,MCO,5.000000,Financials,0.152519,0.306445,1.229417
7,NEM,1.204255,Materials,0.080441,0.363536,0.545576
8,PSX,20.000000,Energy,0.266683,0.403147,0.981680
9,VMC,3.795745,Materials,0.115736,0.316532,1.038715


In [311]:
beta_max_range = np.linspace(0.5, 1.4, 10)  
vol_max_range = np.linspace(0.22, 0.4, 10)

beta_iter = m.addSet('beta_iter', records=beta_max_range)
vol_iter = m.addSet('vol_iter', records=vol_max_range)
results = m.addParameter('results', [beta_iter, vol_iter, 'portfolio_return'])

beta_iters = m.addParameter('beta_iters', [beta_iter], records=beta_max_range)
vol_iters = m.addParameter('vol_iters', [vol_iter], records=vol_max_range)

In [312]:
%%gams
loop(beta_iter,
    beta_max = beta_iters(beta_iter);

    loop(vol_iter,
        vol_max = vol_iters(vol_iter);
        solve portfolio_opt using lp maximizing portfolio_return;
        results(beta_iter, vol_iter, 'expected_return') = portfolio_return.l;
    );
);

--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).
--- Warning: The GAMS version (44.3.0) differs from the API version (45.4.0).


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),0.0980,26,287,LP,CPLEX,0
1,Normal (1),Optimal Global (1),0.0980,26,287,LP,CPLEX,0
2,Normal (1),Optimal Global (1),0.0980,26,287,LP,CPLEX,0
3,Normal (1),Optimal Global (1),0.0980,26,287,LP,CPLEX,0.016
4,Normal (1),Optimal Global (1),0.0980,26,287,LP,CPLEX,0
...,...,...,...,...,...,...,...,...
95,Normal (1),Optimal Global (1),0.1773,26,287,LP,CPLEX,0
96,Normal (1),Optimal Global (1),0.1801,26,287,LP,CPLEX,0
97,Normal (1),Optimal Global (1),0.1823,26,287,LP,CPLEX,0
98,Normal (1),Optimal Global (1),0.1837,26,287,LP,CPLEX,0


In [313]:
results.records

,beta_iter,vol_iter,uni,value
0,0.5,0.22,expected_return,0.097971
1,0.5,0.24,expected_return,0.097971
2,0.5,0.26,expected_return,0.097971
3,0.5,0.28,expected_return,0.097971
4,0.5,0.30000000000000004,expected_return,0.097971
...,...,...,...,...
95,1.4,0.32,expected_return,0.177288
96,1.4,0.34,expected_return,0.180109
97,1.4,0.36,expected_return,0.182267
98,1.4,0.38,expected_return,0.183740


In [17]:
%gams_cleanup --closedown